In [29]:
# Load environment variables from a .env file
from dotenv import load_dotenv
import os

load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [43]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [50]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GOOGLE_API_KEY,
)

In [51]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert technical writer who writes blog posts about technology. You have a good understanding of technology trends but often double check facts and back your conjectures with real factual data. Always give clear, concise, and straight to the point answers."),
        ("user", "{user_input}")
    ]
)

In [52]:
chain = prompt | llm

In [53]:
response = chain.invoke({"user_input", "Who are you?"})

In [54]:
response.content

'I am an expert technical writer who specializes in creating blog posts about technology. I have a strong grasp of current technology trends and a commitment to factual accuracy, always backing information and conjectures with real data.'

In [67]:
class State(dict):
    text: str
    topics: str
    title: str
    keywords: list[str]

In [68]:
# Define nodes
def extract_topics(state: State) -> State:
    prompt = f"Extract 1-3 key topics from the following text:\n\n{state['text']}"
    response = chain.invoke({"user_input", prompt})
    state["topics"] = response.content.strip()
    return state

def generate_title(state:State) -> State:
    prompt = f"Generate two catchy blog titles for each one of these topics:\n\n{state['topics']}"
    response = chain.invoke({"user_input", prompt})
    state["title"] = response.content.strip()
    return state

In [69]:
from langgraph.graph import StateGraph, END

# Build graph
workflow = StateGraph(State)
workflow.add_node("extract_topics", extract_topics)
workflow.add_node("generate_title", generate_title)

# Define node connections
workflow.set_entry_point("extract_topics")
workflow.add_edge("extract_topics", "generate_title")
workflow.add_edge("generate_title", END)

# Compile graph
graph = workflow.compile()

In [70]:
# TEST
input_text = (
    "OpenAI on Thursday launched its GPT-5.2 artificial intelligence model, after CEO Sam Altman reportedly issued an internal 'code red' in early December pausing non‑core projects and redirecting teams to accelerate development in response to Google's Gemini 3.",
    "GPT-5.2 comes with improvements in general intelligence, coding and long-context understanding, the company said in a statement.",
    "The new model is expected to bring even more economic value for users, as it is better at creating spreadsheets, building presentations and handling complex multi-step projects, OpenAI said.",
)

result = graph.invoke({"text": input_text})

In [ ]:
print("Topics:", result['topics'])
print("\n" + "="* 60 + "\n")
print("Suggested Blog Titles:", result['title'])

Topics: Here are 3 key topics from the text:

1.  **OpenAI's GPT-5.2 Launch:** The primary announcement is the release of OpenAI's new AI model, GPT-5.2.
2.  **Competitive Response to Google:** OpenAI's accelerated development was reportedly driven by Google's Gemini 3, prompting an internal "code red."
3.  **GPT-5.2's Enhanced Capabilities and Economic Value:** The new model offers improvements in general intelligence, coding, and long-context understanding, leading to increased economic value through better handling of complex tasks and content creation.
Suggested Blog Titles: Here are two catchy blog titles for each topic:

### 1. OpenAI's GPT-5.2 Launch

*   **GPT-5.2 Has Arrived: OpenAI's Next-Gen AI Unleashed**
*   **Meet GPT-5.2: The New Brain Behind OpenAI's Innovation**

### 2. Competitive Response to Google

*   **Code Red to Market Lead? How Google's Gemini Spurred GPT-5.2**
*   **The AI Race Heats Up: OpenAI's Swift Response to Google's Challenge**

### 3. GPT-5.2's Enhance